In [1]:

class Instruction:
    def __init__(self, instruction):
        self.instructionVal = bin(instruction)
        self.opcode = instruction >> 26 & 0b111111
        self.rs = instruction >> 21 & 0b11111
        self.rt = instruction >> 16 & 0b11111
        self.rd = instruction >> 11 & 0b11111
        self.shamt = instruction >> 6 & 0b11111
        self.funct = instruction & 0b111111
        self.imm = instruction & 0xFFFF

In [7]:
opcodes = {
    0x0 : 'rtype', 0x8: 'addi', 0xd :'ori', 0xe: 'xori', 0xc:'andi', 0xa:'slti',
    0x23: 'lw', 0x2b : 'sw', 0x4:'beq', 0x2:'j', 0x3 : 'jal', 0x5:'bne'
      
}
notWrites = [0x2b, 0x4, 0x2, 0x5]
rtypes = {
    0x20: 'add', 0x22:'sub', 0x24:'and', 0x25: 'or', 0x2a:'slt', 0x26: 'xor', 
    0x27:'nor', 0x0:'sll', 0x2:'srl', 0x8:'jr'
    
}
def signed(data):
        if data & 0x8000:
            return data - (1 << 16)  
        else: return data

In [ ]:
class ROB:
    def __init__(self, register, data):
        self.done = 0
        self.data = data
        self.register = register


In [14]:
class ProcessorState:
    def __init__(self):
        self.cycle = 0
        self.registers = [0] *32
        self.reservationStations = {# the RS is a dictionary with the key being the reservation station number and the value being an instance
        }
        self.ROB = {}#the ROB is a dictionary where the index is the key and the value is an ROB class instance
        self.RAT = {}#RAT simply has two possible values, if 0, then the key indexes the RF. if a value, the value indexes the ROB
        self.units = {#simulate execution units. for example, if we want to have 2 ALUS, we just append two instructions in the ALU array. And then then instructions are handled
            "ALU":[],
            "Load/Store": []
        }
        self.memory = [0] * 1024

In [9]:
class pipelineRegister:
    def __init__(self):
        self.instruction = 0
        self.instructionVal = 0
        self.data = 0 # any data, depending on the stage. signals are interpreted directly w/o a CU so no need for multiple signals 
        self.done = 0 #to ensure sequential execution between the stages
        self.flush = 0#to flush if needed 
        self.thisPC = 0 #the pc of THIS instruction
        self.nextPC = 0 #the pc of the NEXT instruction

In [13]:
class RS:
    def __init__(self, op1, op2):
        self.op1 = op1
        self.op2 = op2
        self.ready = 0

In [11]:
instructions = []
with open("imem.txt", 'r') as inst:
    for line in inst:
        parts = line.split(":")
        if len(parts)>1:
            instructions.append(int(parts[1].strip().rstrip(";"),2))

In [12]:
state = ProcessorState()
def fetch(state, instructions, IF_ID):
    if IF_ID.done:
        return
    elif state.ifidFlush:
        IF_ID.instruction = Instruction(0)
        IF_ID.instructionVal = 0
        inst = Instruction(0)
    else:
        if state.pc < len(instructions):
            inst = Instruction(instructions[state.pc])
        else: inst = Instruction(0)
        IF_ID.instruction = inst
        IF_ID.instructionVal = inst.instructionVal
        IF_ID.pcp1 = state.pc +1
        if opcodes[inst.opcode] == 'beq' or opcodes[inst.opcode] =='bne':
            state.pc = state.pc +1+ signed(inst.imm )& 0b1111111111
        elif opcodes[inst.opcode] == 'j' or opcodes[inst.opcode] =='jal':
            state.pc = inst.imm &0b1111111111
        elif opcodes[inst.opcode] == 'rtype' and rtypes[inst.funct] == 'jr':
            state.pc = state.registers[inst.rs] & 0b1111111111
        else: 
             state.pc += 1
    IF_ID.pc = state.pc
    IF_ID.done = 1
        